In [ ]:
import os
import json
from pprint import pprint
import requests
import pandas as pd
import time
from config import api_key, g_key
import gmaps


In [ ]:
filepath = os.path.join("Top_100_Contractors_Report_Fiscal_Year_2019.xls")


In [ ]:
procurement_data = pd.read_excel(filepath, sheet_name = None)

In [ ]:
print(procurement_data)

In [ ]:
federal_vendor_names = procurement_data["Federal"][["Global Vendor Name","Number of Actions", "Dollars Obligated"]]
federal_vendor_names = federal_vendor_names.rename({"Global Vendor Name": "Business Name"}, axis = 1)
federal_vendor_names

In [ ]:
url = "https://api.data.gov/sam/v3/registrations?qterms="

businesses = federal_vendor_names["Business Name"]
bus_name = []
address_line_one = []
address_city = []
state = []
zip_code = []
country = []                                    
duns = []
plus_four = []
not_found = []
i = 0

for business in businesses:
    query_url = f"{url}{business[0:6]}&length=500&api_key={api_key}"
    response = requests.get(query_url).json()
    i = i+1
    print(f"I'm looking for {business}, number {i} of 100")
    time.sleep(1)
    for result in response["results"]:
        if business == result["legalBusinessName"].upper():
            bus_name.append(result["legalBusinessName"].upper())
            address_line_one.append(result["samAddress"]["line1"])
            address_city.append(result["samAddress"]["city"])
            state.append(result["samAddress"]["stateOrProvince"])
            zip_code.append(result["samAddress"]["zip"])
            country.append(result["samAddress"]["countryCode"])
            duns.append(result["duns"])
            plus_four.append(result["duns_plus4"])
            print(f"I've added information for request: {business}")
            break



In [ ]:
print(len(bus_name))
print(len(address_line_one))
print(len(address_city))
print(len(zip_code))
print(len(country))

In [ ]:
federal_spending = pd.DataFrame({"Business Name":bus_name, "Address": address_line_one, "City": address_city, "State":state, "Zip":zip_code, "Country": country})
federal_spending = pd.merge(federal_spending, federal_vendor_names, on = "Business Name", how = "left")
federal_spending["Lat"] = ""
federal_spending["Lon"] = ""
federal_spending

In [ ]:
cities = federal_spending["State"]+", "+federal_spending["City"]
print(cities)

In [ ]:
cities = federal_spending["City"]+", "+federal_spending["State"]

base_url = "https://maps.googleapis.com/maps/api/geocode/json"
lat = []
lng = []

for city in cities:
    params = {"address": {city}, "key": g_key}
    response = requests.get(base_url, params=params).json()
    lat.append(response["results"][0]["geometry"]["location"]["lat"])
    lng.append(response["results"][0]["geometry"]["location"]["lng"])

In [ ]:
print(lat)

In [ ]:
federal_spending["Lat"] = lat
federal_spending["Lon"] = lng
federal_spending["Duns"] = duns
federal_spending

In [ ]:
gmaps.configure(api_key=g_key)
locations = federal_spending[["Lat", "Lon"]]
weights = federal_spending["Dollars Obligated"]

In [ ]:
fig = gmaps.figure()

dollars_spent = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(dollars_spent)

fig

In [ ]:
info_box_template = """
<dl>
<dt>Company Name</dt><dd>{Business Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>State</dt><dd>{State}</dd>
<dt>Dollars Allocated</dt><dd>{Dollars Obligated}</dd>
</dl>
"""
company_info = [info_box_template.format(**row) for index, row in federal_spending.iterrows()]
locations = federal_spending[["Lat", "Lon"]]

info_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content= company_info
)

fig.add_layer(info_layer)
fig

In [ ]:
print(duns)

In [ ]:
print(plus_four)

In [ ]:
url_duns = "https://api.data.gov/sam/v8/registrations/"
plus_four = "0000"


small_bus = []
businessStartDate = []
naicsName = []

for dun in duns:
    query_url_two = f"{url_duns}{dun}0000?api_key={api_key}"
    response = requests.get(query_url_two).json()
    print(f"I'm getting information for {dun}, {query_url_two}")
    time.sleep(1)
    businessStartDate.append(response["sam_data"]["registration"]["businessStartDate"])
    try:
        attempt_one = str(response["sam_data"]["registration"]["certifications"]).split("'naics': [")[1].split(", {")[0].replace("'",'"').replace("False", '"False"').replace("True", '"True"')
        attempt_two = json.loads(attempt_one)
        small_bus.append(attempt_two["naicsName"])
        naicsName.append(attempt_two["isSmallBusiness"])
    except IndexError:
        print("None for you!")
    
    


In [ ]:
attempt_one = str(response["sam_data"]["registration"]["certifications"]).split("'naics': [")[1].split(", {")[0].replace("'",'"').replace("False", '"False"')
json.loads(attempt_one)

In [ ]:
print(businessStartDate)

In [ ]:
response["sam_data"]["registration"]["certifications"]["farResponses"]

In [ ]:
education_vendor_names = procurement_data["Education (9100)"][["Global Vendor Name", "Number of Actions", "Dollars Obligated"]]
education_vendor_names

In [ ]:
edu_url = "https://api.data.gov/sam/v3/registrations?qterms="

ed_business = procurement_data["Education (9100)"]["Global Vendor Name"]
ed_name = []
ed_address = []
ed_city = []
ed_state = []
ed_zip = []
ed_country = []                                    
ed_duns = []

i = 0

for business in ed_business:
    ed_query_url = f"{edu_url}{business[0:6]}&length=500&api_key={api_key}"
    response = requests.get(ed_query_url).json()
    i = i+1
    print(f"I'm looking for {business}, number {i} of 100")
    time.sleep(1)
    for result in response["results"]:
        if business == result["legalBusinessName"].upper():
            ed_name.append(result["legalBusinessName"].upper())
            ed_address.append(result["samAddress"]["line1"])
            ed_city.append(result["samAddress"]["city"])
            ed_state.append(result["samAddress"]["stateOrProvince"])
            ed_zip.append(result["samAddress"]["zip"])
            ed_country.append(result["samAddress"]["countryCode"])
            ed_duns.append(result["duns"])
            print(f"I've added information for request: {business}")
            break


In [ ]:
print(len(ed_name))